In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('../data/SBAnational.csv')
df.head(1)

/var/folders/41/hgg8f6_j1dj4y4mt0w22d8ym0000gn/T/ipykernel_79717/4085732375.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/SBAnational.csv')


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"


### Target
The target will be binary encoded, 1 representing a defaulted loand and 0 the case of full payment.

In [4]:
# Drop data with null MIS_Status, as it is useless
df.dropna(subset=['MIS_Status'], inplace=True)
df['MIS_Status'].describe()

count     897167
unique         2
top        P I F
freq      739609
Name: MIS_Status, dtype: object

In [5]:
target = 'Default'
# Encode the target variable to binary values. This will encode Default as 1, which in my opinion makes everything more intuitive.
df[target] = np.where(df['MIS_Status'] == 'CHGOFF', 1, 0)
df[target].describe()

# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# df[target] = label_encoder.fit_transform(df['MIS_Status'])
# df.drop(['MIS_Status'], inplace=True, axis=1)
# print(df[target].describe())
# # Paid in Full is encoded as 1

count    897167.000000
mean          0.175617
std           0.380494
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: Default, dtype: float64

In [6]:
# Now let's go over all of these columns
df.columns

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv',
       'Default'],
      dtype='object')

### NAICS

As we're keeping things simple in this first iteration, we will only take a look at the first two digits of NAICS, which encode the broad industry.

In [7]:
## Convert NAICS to a string to make easier getting two digits.
df['NAICS'] = df['NAICS'].astype(str)

## Get only first two digits of NAICS, an convert it back to integer.
def NAICS_conversion(naics_string):
    return int(naics_string[:2])

df['NAICS_i'] = df['NAICS'].apply(NAICS_conversion).astype(int)

In [8]:
df['NAICS_i'].value_counts().head(5)

NAICS_i
0     201667
44     84567
81     72395
54     67922
72     67511
Name: count, dtype: int64

In [9]:
# We are going to create an array of features to be considered in the model.
features = ['NAICS_i']

### Term
We will use term in months as a numerical feature

In [10]:
# Term gets directly used as a numerical feature.
features.append('Term')

### NewExist
We will change it to isNewBusiness, which is 1 when the company is of new creation.

In [11]:
# NewExist has samples with value 0.0 and nulls. Let's drop these.
df = df[(df['NewExist'] != 0) & (df['NewExist'].notnull())]

df['NewExist'].value_counts()

NewExist
1.0    643446
2.0    252559
Name: count, dtype: int64

In [12]:
# Make a column called NewBusiness, which is 0 when NewExist is 1 and 1 when it's 2.

df['isNewBusiness'] = df['NewExist'] - 1
df['isNewBusiness'].describe()

count    896005.000000
mean          0.281872
std           0.449912
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: isNewBusiness, dtype: float64

In [13]:
# Gets added as a feature
features.append('isNewBusiness')

### FranchiseCode
For now, we will use it as isFranchise, being 1 when a business is a franchise and 0 otherwise.

In [14]:
# Function to create the 'isFranchise' column
def is_franchise(code):
    return 0 if code in [0, 1] else 1

df['isFranchise'] = df['FranchiseCode'].apply(is_franchise)

In [15]:
df['isFranchise'].describe()

count    896005.000000
mean          0.057647
std           0.233075
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: isFranchise, dtype: float64

In [16]:
# Gets added to our features
features.append('isFranchise')

### UrbanRural
Each of the three options of UrbanRural (0, 1 and 2) are considered as valid inputs.

In [17]:
df['UrbanRural'] = df['UrbanRural'].astype(str)
df['UrbanRural'].describe()

count     896005
unique         3
top            1
freq      468811
Name: UrbanRural, dtype: object

In [18]:
# Gets added to our features
features.append('UrbanRural')

### RevLineCr
We will reduce it to 1, if it is a Revolving Line of Credit, or 0 otherwise.

In [19]:
df['RevLineCr'].value_counts()

RevLineCr
N    418229
0    257375
Y    200588
T     15232
1        22
R        14
`        11
2         6
C         2
3         1
,         1
7         1
A         1
5         1
.         1
4         1
-         1
Q         1
Name: count, dtype: int64

In [20]:
# A lot of the possible values are junk. We assume that 0 and N are equivalent and drop the rest of outliers.
df['RevLineCr'] = df['RevLineCr'].replace('N', '0')
df['RevLineCr'] = df['RevLineCr'].replace('0', 0)
df['RevLineCr'] = df['RevLineCr'].replace('Y', 1)

In [21]:
df = df[df['RevLineCr'].isin([0, 1])]
df['RevLineCr'] = df['RevLineCr'].astype(int)

In [22]:
df['RevLineCr'].describe()

count    876192.000000
mean          0.228932
std           0.420145
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: RevLineCr, dtype: float64

In [23]:
# Gets added to our features
features.append('RevLineCr')

### LowDoc
Similarly, will be 1 if the loan is part of the LowDoc program and 0 otherwise.

In [24]:
df['LowDoc'].value_counts()

LowDoc
N    762649
Y    107865
0      1238
C       752
S       599
A       491
R        73
1         1
Name: count, dtype: int64

In [25]:
# All values which are not N or Y get dropped.
df['LowDoc'] = df['LowDoc'].replace('N', 0)
df['LowDoc'] = df['LowDoc'].replace('Y', 1)

In [26]:
df = df[df['LowDoc'].isin([0, 1])]
df['LowDoc'] = df['LowDoc'].astype(int)

In [27]:
df['LowDoc'].describe()

count    870514.000000
mean          0.123910
std           0.329479
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: LowDoc, dtype: float64

In [28]:
features.append('LowDoc')

### GrAppv
The size of the loan will be used as a feature after being converted to float numbers.

In [29]:
#Parse dollar values to float value
def dollar_to_float(dollar_string: str):
    return float(dollar_string.replace('$','').replace(',',''))

In [30]:
df['GrAppv'] = df['GrAppv'].apply(dollar_to_float)

In [31]:
df['GrAppv'].describe()

count    8.705140e+05
mean     1.951101e+05
std      2.843112e+05
min      1.000000e+03
25%      3.500000e+04
50%      9.400000e+04
75%      2.300000e+05
max      5.000000e+06
Name: GrAppv, dtype: float64

In [32]:
# Gets added to our features
features.append('GrAppv')

### SBA_Appv (SBARatio)
As SBARatio is a product of GrAppv and SBA_Appv, there is no need in keeping all three. We will stick to SBARatio and GrAppv.

In [33]:
# Convert SBA_Appv to float numbers
df['SBA_Appv'] = df['SBA_Appv'].apply(dollar_to_float)

In [34]:
# Compute the ratio as the division between the two.
df['SBARatio'] = df['SBA_Appv'] / df['GrAppv']

In [37]:
df['SBARatio'].describe()

count    870514.000000
mean          0.711526
std           0.173216
min           0.028140
25%           0.500000
50%           0.750000
75%           0.850000
max           1.000000
Name: SBARatio, dtype: float64

In [36]:
# Gets added to our features
features.append('SBARatio')

In [38]:
df[features].describe()

,NAICS_i,Term,isNewBusiness,isFranchise,RevLineCr,LowDoc,GrAppv,SBARatio
count,870514.000000,870514.000000,870514.000000,870514.000000,870514.000000,870514.000000,8.705140e+05,870514.000000
mean,39.514401,111.508946,0.282303,0.058415,0.229363,0.123910,1.951101e+05,0.711526
std,26.364679,79.177488,0.450120,0.234526,0.420424,0.329479,2.843112e+05,0.173216
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+03,0.028140
25%,23.000000,60.000000,0.000000,0.000000,0.000000,0.000000,3.500000e+04,0.500000
50%,44.000000,84.000000,0.000000,0.000000,0.000000,0.000000,9.400000e+04,0.750000
75%,56.000000,120.000000,1.000000,0.000000,0.000000,0.000000,2.300000e+05,0.850000
max,92.000000,569.000000,1.000000,1.000000,1.000000,1.000000,5.000000e+06,1.000000


In [39]:
df.describe()

,LoanNr_ChkDgt,Zip,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,GrAppv,SBA_Appv,Default,NAICS_i,isNewBusiness,isFranchise,SBARatio
count,8.705140e+05,870514.000000,870514.000000,870514.000000,870514.000000,870514.000000,870514.000000,870514.000000,870514.000000,870514.000000,8.705140e+05,8.705140e+05,870514.000000,870514.000000,870514.000000,870514.000000,870514.000000
mean,4.770738e+09,53958.314583,111.508946,11.479750,1.282303,8.612161,10.969103,2794.073226,0.229363,0.123910,1.951101e+05,1.514752e+05,0.171462,39.514401,0.282303,0.058415,0.711526
std,2.534638e+09,31138.741113,79.177488,74.589471,0.450120,239.964650,240.424854,12847.026352,0.420424,0.329479,2.843112e+05,2.288713e+05,0.376912,26.364679,0.450120,0.234526,0.173216
min,1.000014e+09,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+03,5.000000e+02,0.000000,0.000000,0.000000,0.000000,0.028140
25%,2.593918e+09,27882.000000,60.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,3.500000e+04,2.240000e+04,0.000000,23.000000,0.000000,0.000000,0.500000
50%,4.360599e+09,55428.000000,84.000000,4.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,9.400000e+04,6.375000e+04,0.000000,44.000000,0.000000,0.000000,0.750000
75%,6.872071e+09,83706.000000,120.000000,10.000000,2.000000,1.000000,4.000000,1.000000,0.000000,0.000000,2.300000e+05,1.770000e+05,0.000000,56.000000,1.000000,0.000000,0.850000
max,9.996003e+09,99999.000000,569.000000,9999.000000,2.000000,8800.000000,9500.000000,99999.000000,1.000000,1.000000,5.000000e+06,4.500000e+06,1.000000,92.000000,1.000000,1.000000,1.000000


In [41]:
df.to_csv('../data/df_clean.csv', index=False)